In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import torch
from datasets import Dataset

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model_name = "facebook/opt-350m"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    device_map="auto"   # Automatically distribute the model across available GPUs
)

# Freeze most parameters in the base model but unfreeze the last few layers for fine-tuning
for name, param in model.named_parameters():
    if "decoder.layers" in name and any(str(i) in name for i in range(22, 24)):  # Unfreeze last two layers (example: layers 22 and 23)
        param.requires_grad = True
    else:
        param.requires_grad = False

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load dataset using kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

# Assuming the dataset is in CSV format
# The file is typically named "spam.csv" in this dataset
df_sms = pd.read_csv(f"{path}/spam.csv", encoding="ISO-8859-1")

# Rename columns to match expected names
df_sms = df_sms.rename(columns={"v1": "label", "v2": "text"})  # 'v1' is label, 'v2' is the message

# Drop unnecessary columns if present
df_sms = df_sms[['text', 'label']]

# Ensure consistent label encoding: 'ham' -> 0, 'spam' -> 1
df_sms['label'] = df_sms['label'].map({'ham': 0, 'spam': 1})

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df_sms)

# Split the dataset into training and validation sets (90% train, 10% validation)
train_data = dataset.select([i for i in range(len(dataset)) if i % 10 != 0])  # 90% for training
val_data = dataset.select([i for i in range(len(dataset)) if i % 10 == 0])  # 10% for validation

# Tokenize the input sequences
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', max_length=128, truncation=True)

# Apply tokenization to the datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Convert labels to a format suitable for classification
train_data = train_data.rename_column("label", "labels")
val_data = val_data.rename_column("label", "labels")

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results_sms_spam",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

# Start Training
trainer.train()

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5014 [00:00<?, ? examples/s]

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/942 [00:00<?, ?it/s]

...

{'eval_loss': 0.2882041931152344, 'eval_runtime': 2.0347, 'eval_samples_per_second': 274.241, 'eval_steps_per_second': 17.201, 'epoch': 0.03}
{'loss': 0.265, 'grad_norm': 1.8173232078552246, 'learning_rate': 1.9575371549893843e-05, 'epoch': 0.06}


  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 0.09563983976840973, 'eval_runtime': 2.0554, 'eval_samples_per_second': 271.48, 'eval_steps_per_second': 17.028, 'epoch': 0.06}
{'loss': 0.084, 'grad_norm': 0.6620710492134094, 'learning_rate': 1.9363057324840767e-05, 'epoch': 0.1}


  0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
# Testing the model
def test_model(test_texts):
    # Tokenize the test texts
    inputs = tokenizer(test_texts, padding='max_length', max_length=128, truncation=True, return_tensors="pt").to(device)
    
    # Get predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.cpu().numpy()

# Example test cases
test_texts = [
    "Free entry in 2 a weekly competition to win FA Cup final tickets! Text FA to 87121 to receive entry question.",
    "Hey, are we still meeting for coffee later?",
    "You have won a $1000 Walmart gift card. Call now to claim your prize.",
    "Can you send me the report by tonight?"
]

# Get predictions
predictions = test_model(test_texts)

# Map predictions back to labels
label_map = {0: "ham", 1: "spam"}
results = [label_map[pred] for pred in predictions]

# Print results
for text, label in zip(test_texts, results):
    print(f"Text: {text}\nPredicted Label: {label}\n")

Text: Free entry in 2 a weekly competition to win FA Cup final tickets! Text FA to 87121 to receive entry question.
Predicted Label: spam

Text: Hey, are we still meeting for coffee later?
Predicted Label: ham

Text: You have won a $1000 Walmart gift card. Call now to claim your prize.
Predicted Label: spam

Text: Can you send me the report by tonight?
Predicted Label: ham



In [4]:
ds

NameError: name 'ds' is not defined

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from datasets import Dataset

# Load the model
model_name = "facebook/opt-350m"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=28,  # Number of unique emotion classes
    device_map="auto"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Freeze most parameters in the base model but unfreeze the last few layers for fine-tuning
for name, param in model.named_parameters():
    if "decoder.layers" in name and any(str(i) in name for i in range(22, 24)):  # Unfreeze last two layers (example: layers 22 and 23)
        param.requires_grad = True
    else:
        param.requires_grad = False

# Load dataset
ds = load_dataset("google-research-datasets/go_emotions", "simplified")

# Convert to Pandas DataFrame for preprocessing
df_train = pd.DataFrame(ds["train"])

# Retain only the required columns
df_train = df_train[['text', 'labels']]

# Flatten the labels if they are lists
df_train['labels'] = df_train['labels'].apply(lambda x: x[0] if isinstance(x, list) else x)

# Split the dataset (20% train, 80% test)
train_subset = df_train.sample(frac=0.2, random_state=42)  # Random 20% for training
test_subset = df_train.drop(train_subset.index)            # Remaining 80% for testing

# Convert back to Hugging Face Dataset
train_data = Dataset.from_pandas(train_subset)
test_data = Dataset.from_pandas(test_subset)

# Tokenize the input sequences
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', max_length=128, truncation=True)

# Apply tokenization
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results_emotions",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

# Start Training
trainer.train()


Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8682 [00:00<?, ? examples/s]

Map:   0%|          | 0/34728 [00:00<?, ? examples/s]

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/1629 [00:00<?, ?it/s]

...

{'eval_loss': 1.6409125328063965, 'eval_runtime': 128.5262, 'eval_samples_per_second': 270.202, 'eval_steps_per_second': 16.891, 'epoch': 2.67}
{'loss': 1.2578, 'grad_norm': 18.194576263427734, 'learning_rate': 2.0748925721301415e-06, 'epoch': 2.69}


  0%|          | 0/2171 [00:00<?, ?it/s]

{'eval_loss': 1.6431468725204468, 'eval_runtime': 129.3176, 'eval_samples_per_second': 268.548, 'eval_steps_per_second': 16.788, 'epoch': 2.69}
{'loss': 1.2846, 'grad_norm': 15.103840827941895, 'learning_rate': 1.9521178637200737e-06, 'epoch': 2.71}


  0%|          | 0/2171 [00:00<?, ?it/s]

: 

In [ ]:
import matplotlib.pyplot as plt

# Access the training log history
training_history = trainer.state.log_history

# Extract the training loss and evaluation loss
training_loss = [log['loss'] for log in training_history if 'loss' in log]
eval_loss = [log['eval_loss'] for log in training_history if 'eval_loss' in log]

# Extract the steps where metrics were logged
steps = [log['step'] for log in training_history if 'loss' in log]

# Plot the training and evaluation loss
plt.figure(figsize=(10, 6))
plt.plot(steps, training_loss, label='Training Loss', marker='o')
if eval_loss:  # Only plot if evaluation loss is logged
    plt.plot(steps, eval_loss[:len(steps)], label='Evaluation Loss', marker='x')

plt.title('Training and Evaluation Loss Over Steps')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()
